<a href="https://colab.research.google.com/github/01Wood/SensenWang/blob/main/assignment3_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###3.7 Neural Networks TensorFlow 2
Here we provide a simple example of Neural Network using TensorFlow 2 with MNIST dataset.

Source:

https://www.tensorflow.org/tutorials/quickstart/beginner

In [1]:
import tensorflow as tf

##1. Load and prepare the MNIST dataset
Reference:

https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data

This is a dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images. It is considered another classic dataset in machine learning area. The images are handwritings of English letters. Researchers are using it as a training set to build neural network models that would recognize human handwritings.

The dataset we import from tf.keras is not the original image set, but numerical matrix that transfromed from the images via certain kernels.

tf.keras.datasets.mnist.load_data() will return tuples of arrays:

* x_train: uint8 NumPy array of grayscale image data with shapes (60000, 28, 28), containing the training data. Pixel values range from 0 to 255.

* y_train: uint8 NumPy array of digit labels (integers in range 0-9) with shape (60000,) for the training data.

* x_test: uint8 NumPy array of grayscale image data with shapes (10000, 28, 28), containing the test data. Pixel values range from 0 to 255.

* y_test: uint8 NumPy array of digit labels (integers in range 0-9) with shape (10000,) for the test data.




In [2]:
# load the dataset
mnist = tf.keras.datasets.mnist

# convert the samples from integers to floating-point numbers
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


## 2. Build the tf.keras.Sequential model by stacking layers.

Choose an optimizer and loss function for training:

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.2627324 ,  0.03959125,  0.21992244, -0.3401888 ,  0.62115407,
        -0.294676  , -0.05768569, -0.08771984, -0.37523183,  0.34173158]],
      dtype=float32)

The tf.nn.softmax function converts these logits to "probabilities" for each class:

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.11976259, 0.09581031, 0.11474377, 0.06553539, 0.17138854,
        0.06858701, 0.08692914, 0.08435712, 0.0632786 , 0.1296075 ]],
      dtype=float32)

The losses.SparseCategoricalCrossentropy loss takes a vector of logits and a True index and returns a scalar loss for each example.

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.6796522

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The Model.fit method adjusts the model parameters to minimize the loss:

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2946 - accuracy: 0.9150
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1424 - accuracy: 0.9569
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1084 - accuracy: 0.9673
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0866 - accuracy: 0.9733
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0770 - accuracy: 0.9754


## 3. Evaluate the Model
The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".

In [10]:

model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0767 - accuracy: 0.9766 - 562ms/epoch - 2ms/step


[0.07668141275644302, 0.9765999913215637]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])


<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.23594987e-06, 3.56382546e-09, 1.01924411e-06, 2.91660865e-04,
        3.97307937e-10, 1.61816442e-05, 1.07635537e-12, 9.99484539e-01,
        2.29895545e-06, 2.03017014e-04],
       [7.68936133e-08, 2.30067126e-05, 9.99976277e-01, 2.29089963e-07,
        5.81999401e-13, 4.77256634e-08, 5.61400215e-09, 3.97386247e-12,
        3.17605554e-07, 3.97265554e-13],
       [2.58415547e-08, 9.98687685e-01, 1.29411826e-04, 1.28368242e-06,
        3.43678839e-05, 3.02423700e-06, 1.27242527e-06, 1.07381423e-03,
        6.84508559e-05, 7.02746888e-07],
       [9.99957561e-01, 2.32535080e-09, 6.12763188e-06, 2.69289551e-08,
        9.06808623e-07, 4.50993230e-06, 2.65620911e-05, 1.07662927e-07,
        1.59124536e-08, 4.11412884e-06],
       [1.31302059e-07, 3.61872288e-09, 1.56623864e-05, 2.07263682e-07,
        9.87157643e-01, 3.89636079e-06, 4.60280580e-06, 1.92498686e-04,
        2.21047299e-06, 1.26231145e-02]], dtype=float32)>